In [26]:
from collections import Counter
import random
from konlpy.tag import Okt
import json

# 문서 d의 모든 단어 가운데 topic에 속하는
# 단어의 비율 (alpha를 더해 smoothing)
def p_topic_given_document(topic, d, alpha=0.1):
    return ((document_topic_counts[d][topic] + alpha) /
            (document_lengths[d] + K * alpha))

# topic에 속한 단어 가운데 word의 비율
# (beta를 더해 smoothing)
def p_word_given_topic(word, topic, beta=0.1):
    return ((topic_word_counts[topic][word] + beta) /
            (topic_counts[topic] + V * beta))

# 문서와 문서의 단어가 주어지면
# k번째 토픽의 weight를 반환
def topic_weight(d, word, k):
    return p_word_given_topic(word, k) * p_topic_given_document(k, d)

def choose_new_topic(d, word):
    return sample_from([topic_weight(d, word, k) for k in range(K)])

def sample_from(weights):
    # i를 weights[i] / sum(weights)
    # 확률로 반환
    total = sum(weights)
    # 0과 total 사이를 균일하게 선택
    rnd = total * random.random()
    # 아래 식을 만족하는 가장 작은 i를 반환
    # weights[0] + ... + weights[i] >= rnd
    for i, w in enumerate(weights):
        rnd -= w
        if rnd <= 0:
            return i
        
class DoublespaceLineCorpus:    
    def __init__(self, corpus_fname, iter_sent = False):
        self.corpus_fname = corpus_fname
        self.iter_sent = iter_sent
            
    def __iter__(self):
        with open(self.corpus_fname, encoding='utf-8') as f:
            if self.iter_sent:
                json_data = json.load(f)
                for doc in json_data:
                    json_content = doc["doc_content"]
                    yield [json_content]
            else:
                json_data = json.load(f)
                for doc in json_data:
                    json_content = doc["doc_content"]
                    yield [json_content.strip()]
                    
corpus_path = 'C:\ex/와이즈넛/주제어 추출/cafe_data/D-00-201908060736-07977.json'                    
corpus = DoublespaceLineCorpus(corpus_path, iter_sent=False)
    #corpus = json_open(corpus_path)
okt = Okt()
T=[]
for cont in corpus:
    T.append(cont)
t =[]
for i in T:
    t.append(str(i))      

        
e = []
for i in range(0,len(t)):
    e.append(okt.nouns(t[i]))
    
documents = e[0:10]

random.seed(0)

# topic 수 지정
K=4

# 각 단어를 임의의 토픽에 랜덤 배정
document_topics = [[random.randrange(K) for word in document]
                    for document in documents]

# 각 토픽이 각 문서에 할당되는 횟수
# Counter로 구성된 리스트
# 각 Counter는 각 문서를 의미
document_topic_counts = [Counter() for _ in documents]

# 각 단어가 각 토픽에 할당되는 횟수
# Counter로 구성된 리스트
# 각 Counter는 각 토픽을 의미
topic_word_counts = [Counter() for _ in range(K)]

# 각 토픽에 할당되는 총 단어수
# 숫자로 구성된 리스트
# 각각의 숫자는 각 토픽을 의미함
topic_counts = [0 for _ in range(K)]

# 각 문서에 포함되는 총 단어수
# 숫자로 구성된 리스트
# 각각의 숫자는 각 문서를 의미함
document_lengths = [len(document) for document in documents]

# 단어 종류의 수
distinct_words = set(word for document in documents for word in document)
V = len(distinct_words)


D = len(documents)

for d in range(D):
    for word, topic in zip(documents[d], document_topics[d]):
        document_topic_counts[d][topic] += 1
        topic_word_counts[topic][word] += 1
        topic_counts[topic] += 1

for iter in range(1000):
    for d in range(D):
        for i, (word, topic) in enumerate(zip(documents[d],
                                              document_topics[d])):
            document_topic_counts[d][topic] -= 1
            topic_word_counts[topic][word] -= 1
            topic_counts[topic] -= 1
            document_lengths[d] -= 1
            new_topic = choose_new_topic(d, word)
            document_topics[d][i] = new_topic
            document_topic_counts[d][new_topic] += 1
            topic_word_counts[new_topic][word] += 1
            topic_counts[new_topic] += 1
            document_lengths[d] += 1

print(topic_word_counts[0])
print(topic_word_counts[1])


Counter({'상가': 17, '매매': 15, '아파트': 14, '가능': 6, '층': 6, '면적': 6, '층수': 5, '전': 5, '춘천시': 5, '보기': 5, '길찾기': 5, '인수': 5, '임대': 5, '특징': 5, '조건': 5, '연락': 5, '공인': 5, '사무소': 5, '기타': 5, '지명': 5, '구분': 5, '월세': 5, '금액': 5, '중개사': 5, '품목': 5, '주택': 5, '주소': 5, '평': 5, '매도': 5, '방문': 5, '강남동': 5, '크게': 5, '매물': 5, '강원도': 5, '부동산': 5, '주차': 4, '방': 4, '내부': 4, '화장실': 4, '단지': 4, '인테리어': 3, '바닥': 3, '효자동': 3, '내': 3, '권리금': 3, '약': 3, '시공': 2, '몸': 2, '앞': 2, '강대': 2, '인구': 2, '전기': 2, '광고': 2, '조명': 2, '업종': 2, '타입': 2, '퇴계동': 2, '부가세': 2, '유동': 2, '시설': 2, '판넬': 2, '전세': 2, '모든': 2, '변': 2, '효과': 2, '수도': 2, '최적화': 1, '골목': 1, '주변': 1, '물': 1, '삼거리': 1, '수': 1, '분리': 1, '및': 1, '량': 1, '벽지': 1, '미용': 1, '매우': 1, '공': 1, '사장': 1, '혁신': 1, '타일': 1, '천정': 1, '상권': 1, '자유': 1, '외부': 1, '정문': 1, '설치': 1, '남여': 1, '창고': 1, '난방': 1, '석사동': 1, '야외': 1, '데크': 1, '그대로': 1, '초등학교': 1, '바로': 1, '포함': 1, '대로': 1, '의류': 1, '대비': 1, '심': 1, '조금': 1, '남향': 1, '영업': 1, '공원': 1, '차량': 1, '통행': 1, '기본': 1, '

In [27]:
document_topic_counts[0]

Counter({3: 0, 0: 69, 2: 0, 1: 0})

In [32]:
document_topic_counts[4]

Counter({2: 0, 0: 57, 3: 0, 1: 0})